<a href="https://colab.research.google.com/github/sakathia/Data110/blob/main/Week_11_Assignment_Sandra_Aching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Week 11 Assignment

- Create a combined heatmap visualization for the **BPD_Arrests.csv** dataset, with at least two interactive layers based on **Gender** (e.g., one layer for arrests involving males and another for arrests involving females).
- Reflect on the patterns revealed in each gender-based layer and discuss any potential insights or societal implications.
- Present your findings by explaining how the spatial distribution differs or overlaps between the two layers.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import folium
from folium.plugins import HeatMap


In [10]:
df = pd.read_csv('/content/BPD_Arrests (1).csv')

In [11]:
df.head()

,X,Y,RowID,IncidentNumber,ArrestNumber,Age,Gender,Race,ArrestDateTime,ArrestLocation,IncidentOffence,IncidentLocation,Charge,ChargeDescription,District,Post,Neighborhood,Latitude,Longitude,GeoLocation
0,1.413638e+06,597194.151583,1,23L09552,23192623.0,16.0,M,B,2023/12/31 21:00:00+00,1500 SCHOOL ST,Unknown Offense,1500 SCHOOL ST,1 1106,HGV,Western,711.0,Sandtown-Winchester,39.3059,-76.642,"(39.3059,-76.642)"
1,1.413638e+06,597194.151583,2,23L09552,23192626.0,21.0,M,B,2023/12/31 21:00:00+00,1500 SCHOOL ST,Unknown Offense,1500 SCHOOL ST,1 1106,HGV,Western,711.0,Sandtown-Winchester,39.3059,-76.642,"(39.3059,-76.642)"
2,1.413638e+06,597194.151583,3,23L09552,24000002.0,18.0,M,B,2023/12/31 21:00:00+00,1500 SCHOOL ST,Unknown Offense,1500 SCHOOL ST,1 1106,HGV,Western,711.0,Sandtown-Winchester,39.3059,-76.642,"(39.3059,-76.642)"
3,1.413638e+06,597194.151583,4,23L09552,24000010.0,19.0,M,B,2023/12/31 21:00:00+00,1500 SCHOOL ST,Unknown Offense,1500 SCHOOL ST,1 5285,HGV,Western,711.0,Sandtown-Winchester,39.3059,-76.642,"(39.3059,-76.642)"
4,NaN,NaN,5,NaN,23192609.0,41.0,F,B,2023/12/31 20:35:00+00,NaN,Unknown Offense,NaN,1 1415,ASSAULT-SEC DEGREE,NaN,NaN,NaN,NaN,NaN,"(,)"


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377824 entries, 0 to 377823
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   X                  227117 non-null  float64
 1   Y                  227117 non-null  float64
 2   RowID              377824 non-null  int64  
 3   IncidentNumber     230349 non-null  object 
 4   ArrestNumber       360643 non-null  float64
 5   Age                377692 non-null  float64
 6   Gender             377790 non-null  object 
 7   Race               377730 non-null  object 
 8   ArrestDateTime     377824 non-null  object 
 9   ArrestLocation     233192 non-null  object 
 10  IncidentOffence    377824 non-null  object 
 11  IncidentLocation   233192 non-null  object 
 12  Charge             328571 non-null  object 
 13  ChargeDescription  377824 non-null  object 
 14  District           219039 non-null  object 
 15  Post               219039 non-null  float64
 16  Ne

In [13]:
cleaned_df = df.dropna(subset=['Latitude', 'Longitude'])

In [14]:
cleaned_df.head()

,X,Y,RowID,IncidentNumber,ArrestNumber,Age,Gender,Race,ArrestDateTime,ArrestLocation,IncidentOffence,IncidentLocation,Charge,ChargeDescription,District,Post,Neighborhood,Latitude,Longitude,GeoLocation
0,1.413638e+06,597194.151583,1,23L09552,23192623.0,16.0,M,B,2023/12/31 21:00:00+00,1500 SCHOOL ST,Unknown Offense,1500 SCHOOL ST,1 1106,HGV,Western,711.0,Sandtown-Winchester,39.3059,-76.6420,"(39.3059,-76.642)"
1,1.413638e+06,597194.151583,2,23L09552,23192626.0,21.0,M,B,2023/12/31 21:00:00+00,1500 SCHOOL ST,Unknown Offense,1500 SCHOOL ST,1 1106,HGV,Western,711.0,Sandtown-Winchester,39.3059,-76.6420,"(39.3059,-76.642)"
2,1.413638e+06,597194.151583,3,23L09552,24000002.0,18.0,M,B,2023/12/31 21:00:00+00,1500 SCHOOL ST,Unknown Offense,1500 SCHOOL ST,1 1106,HGV,Western,711.0,Sandtown-Winchester,39.3059,-76.6420,"(39.3059,-76.642)"
3,1.413638e+06,597194.151583,4,23L09552,24000010.0,19.0,M,B,2023/12/31 21:00:00+00,1500 SCHOOL ST,Unknown Offense,1500 SCHOOL ST,1 5285,HGV,Western,711.0,Sandtown-Winchester,39.3059,-76.6420,"(39.3059,-76.642)"
8,1.411743e+06,589647.105193,9,23L09538,23192606.0,20.0,M,B,2023/12/31 18:53:00+00,2000 PRATT ST,Unknown Offense,2000 PRATT ST,1 1785,HGV,Southwest,832.0,Carrollton Ridge,39.2852,-76.6488,"(39.2852,-76.6488)"


In [18]:
#Filtering the data for Gender to create the layers for the combined heat map
Male_Arrest_BPD = cleaned_df[cleaned_df['Gender'] == 'M']
Female_Arrest_BPD = cleaned_df[cleaned_df['Gender'] == 'F']

In [17]:
# Create a base map centered around Baltimore, Maryland
map_center = [39.2904, -76.6122]  # Center of Baltimore
combined_map = folium.Map(location=map_center, zoom_start=11)



In [20]:
#Prepairing the Coordinates for the layers
Male_Arrest_BPD_data = Male_Arrest_BPD[['Latitude', 'Longitude']].values
Female_Arrest_BPD_data = Female_Arrest_BPD[['Latitude', 'Longitude']].values

In [21]:
#Creating the Layers and feature groups for each heatmap layer
Male_Arrest_BPD_layer = folium.FeatureGroup(name="Male Arrests Heatmap")
Female_Arrest_BPD_layer = folium.FeatureGroup(name="Female Arrests Heatmap")

In [23]:
# Adding the heatmap layers to each feature group
HeatMap(Male_Arrest_BPD_data).add_to(Male_Arrest_BPD_layer)
HeatMap(Female_Arrest_BPD_data).add_to(Female_Arrest_BPD_layer)

In [24]:
# Adding the feature groups to the combined map
Male_Arrest_BPD_layer.add_to(combined_map)
Female_Arrest_BPD_layer.add_to(combined_map)

In [25]:
# Adding the layer control to toggle the layers on and off
folium.LayerControl().add_to(combined_map)

# Saving the combined map to an HTML file for viewing
combined_map_path = "BPD_combined_map_layers1.html"
combined_map.save(combined_map_path)

# Displaying path for downloading
combined_map_path

'BPD_combined_map_layers1.html'

## Reflection/Findings

I was able to create heatmap layers based on Gender for BPD Arrests. In those layers I was able to see that in terms of Gender, there was more Males Arrests covering the Baltimore area than Female Arrests. Also, there was more concentration of Arrests seen for Males than there were for Females. I believe the difference can be due to the different crime types committed by each Gender. Socioeconomic status could also be an indicator to the type of crime committed by Gender for example, if they are theft related. Also, since the concentration is higher for Males than Females, I would also note that there may be some policing biases in targeting Males Suspects for crime types and may not be focusing the same attention on Female Suspects for similar crime types when conducting investigations and Arrests. Then, the overlap that I see in both Genders for Arrests can be due to some areas either having lots of traffic in terms of stores, markets or transporation Bus/Metro stops being more likely for crimes and arrests to occur. Also, overlaps seen between genders can also be noted in the distribtution in the heatmap by socioeconomic status as some neighborhoods that are deemed poor may see more concentration of crimes committed and Arrests than more affluent neighborhoods.